In [1]:
import cbpro as gdax
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict

from datetime import datetime, timedelta
from math import ceil
from time import sleep

%matplotlib inline
client = gdax.PublicClient()

In [43]:
def historic_data(client, product, num_candles, granularity, max_per=300):
    start = datetime.utcnow().replace(microsecond=0)
    delta = timedelta(seconds=granularity) * max_per
    full = ceil(num_candles / max_per)
    rates = []
    for i in range(1, full + 2):
        frm = start - (delta * i)
        to  = frm + delta
        print('From: {}, To: {}'.format(frm.isoformat(), to.isoformat()))
        chunk = client.get_product_historic_rates(
            product, start=frm.isoformat(), end=to.isoformat(), granularity=granularity)
        if type(chunk) is not list:
            print(chunk)
        else:
            rates.extend(chunk)
        sleep(0.5)
    return rates 

In [56]:
def save_rates(product, num_candles=None, granularity=60):
    rates = historic_data(client=client, product=product, num_candles=num_candles, granularity=granularity)
    print('{} prices'.format(len(rates)))
    # Load into Pandas
    pd_rates = pd.DataFrame(rates)
    # Set column names
    pd_rates.columns = ["Time", "Low", "High", "Open", "Close", "Volume"]
    # Convert Unix time to UTC
    pd_rates["Time"] = pd.to_datetime(pd_rates["Time"],unit='s')
    # Remove duplicates
    before = len(pd_rates)
    pd_rates.drop_duplicates(inplace=True)
    print("Dropped {} duplicate rows".format(before - len(pd_rates)))
    print(pd_rates.head())
    # Save to CSV
    fn = "./historic_data/{}_{}_{}.csv".format(product, granularity, num_candles)
    pd_rates.to_csv(fn, index=False)
    return pd_rates


In [ ]:
prods = client.get_products()
for prod in prods:
    save_rates(prod['id'], granularity=60, num_candles=300)

From: 2018-10-28T11:54:00, To: 2018-10-28T16:54:00
From: 2018-10-28T06:54:00, To: 2018-10-28T11:54:00
124 prices
Dropped 0 duplicate rows
                 Time   Low  High  Open  Close  Volume
0 2018-10-28 16:48:00  8.38  8.38  8.38   8.38    5.86
1 2018-10-28 16:43:00  8.39  8.39  8.39   8.39    3.80
2 2018-10-28 16:42:00  8.39  8.39  8.39   8.39   11.40
3 2018-10-28 16:41:00  8.39  8.39  8.39   8.39    5.69
4 2018-10-28 16:36:00  8.39  8.39  8.39   8.39   11.40
From: 2018-10-28T11:54:02, To: 2018-10-28T16:54:02
From: 2018-10-28T06:54:02, To: 2018-10-28T11:54:02
600 prices
Dropped 0 duplicate rows
                 Time      Low     High     Open    Close  Volume
0 2018-10-28 16:53:00  6402.05  6402.58  6402.58  6402.58   0.223
1 2018-10-28 16:52:00  6401.21  6403.00  6402.76  6402.64   0.585
2 2018-10-28 16:51:00  6401.23  6402.82  6401.24  6402.75   1.180
3 2018-10-28 16:50:00  6401.10  6402.93  6402.31  6401.21   1.710
4 2018-10-28 16:49:00  6401.09  6402.88  6402.44  6402.15   0.99

From: 2018-10-28T06:54:22, To: 2018-10-28T11:54:22
188 prices
Dropped 0 duplicate rows
                 Time       Low      High      Open     Close  Volume
0 2018-10-28 16:53:00  0.000125  0.000125  0.000125  0.000125    3.00
1 2018-10-28 16:48:00  0.000125  0.000125  0.000125  0.000125  320.58
2 2018-10-28 16:47:00  0.000125  0.000125  0.000125  0.000125  305.90
3 2018-10-28 16:46:00  0.000125  0.000125  0.000125  0.000125  219.81
4 2018-10-28 16:45:00  0.000125  0.000125  0.000125  0.000125  191.81
From: 2018-10-28T11:54:23, To: 2018-10-28T16:54:23
From: 2018-10-28T06:54:23, To: 2018-10-28T11:54:23
344 prices
Dropped 0 duplicate rows
                 Time  Low  High  Open  Close  Volume
0 2018-10-28 16:52:00  0.7   0.7   0.7    0.7   26.30
1 2018-10-28 16:50:00  0.7   0.7   0.7    0.7  122.23
2 2018-10-28 16:49:00  0.7   0.7   0.7    0.7   83.96
3 2018-10-28 16:48:00  0.7   0.7   0.7    0.7   85.84
4 2018-10-28 16:47:00  0.7   0.7   0.7    0.7  610.40
From: 2018-10-28T11:54:25, To: 